# Santander

# 1. Import libs
[top](#Contents)

In [1]:
# pandas and numpy imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.metrics')

In [27]:
# pip install -U imbalanced-learn

# 2. Get data
[top](#Contents)

#### Read data
[top](#Contents)

In [2]:
bank_ds = pd.read_csv('train.csv')

In [14]:
bank_test_ds = pd.read_csv('test.csv')

#### Drop an unneeded column

In [3]:
# Assuming 'column_to_drop' is the name of the column you want to drop
column_to_drop = 'ID_code'

# Drop the column from bank_ds
bank_ds.drop(column_to_drop, axis=1, inplace=True)


### Split 70/30 (stratified sampling)

In [4]:
target = bank_ds['target']

In [5]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    bank_ds.drop(labels=['target'], axis=1),  # drop the target
    bank_ds['target'],  # just the target
    test_size=0.3,
    stratify=target,
    random_state=42)

X_train.shape, X_test.shape

((140000, 200), (60000, 200))

### Oversampling

In [6]:
from imblearn.over_sampling import (
    RandomOverSampler,
    SMOTE,
    ADASYN,
    BorderlineSMOTE,
    SVMSMOTE,
)
from sklearn.svm import SVC

In [7]:
oversampler_dict = {
    'random': RandomOverSampler(sampling_strategy='auto', random_state=0),
    'smote': SMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, n_jobs=-1),
    'adasyn': ADASYN(sampling_strategy='auto', random_state=0, n_neighbors=5, n_jobs=4),
    'border1': BorderlineSMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, m_neighbors=10, kind='borderline-1', n_jobs=4),
    'border2': BorderlineSMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, m_neighbors=10, kind='borderline-2', n_jobs=4),
}

In [19]:
#oversampler_dict = {
    'random': RandomOverSampler(sampling_strategy='auto', random_state=0),
    'smote': SMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, n_jobs=-1),
    'adasyn': ADASYN(sampling_strategy='auto', random_state=0, n_neighbors=5, n_jobs=4),
    'border1': BorderlineSMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, m_neighbors=10, kind='borderline-1', n_jobs=4),
    'border2': BorderlineSMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, m_neighbors=10, kind='borderline-2', n_jobs=4),
    'svm': SVMSMOTE(sampling_strategy='auto', random_state=0, k_neighbors=5, m_neighbors=10, n_jobs=4, svm_estimator=SVC(kernel='linear'))
}


## Evaluation

#### Metrics which require predict_proba method
- The ROC AUC score is calculated based on the predicted probabilities for each class, which are obtained using the predict_proba method. This method provides the probability estimates for each class, which are necessary to compute the ROC curve and subsequently the AUC (Area Under the Curve).

- make_scorer is used with roc_auc_score and average_precision_score, with the needs_proba=True argument to ensure that predict_proba is used for these metrics.
- The cross_validate function internally uses predict_proba to get the predicted probabilities required for calculating ROC-AUC and PR-AUC

- Over-sampling methods should be performed on the dataset that we are going to use to train the classifier. But, the performance of the model should be determined on a portion of the data, that was not re-sampled

In [8]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import make_pipeline

# Dictionary to hold the models
models_dict = {
    #'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, solver='lbfgs', max_iter=1000, verbose=0),  # Suppress logs 
   # 'XGBoost': XGBClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42, verbose=-1),  # Suppress LightGBM logs
    'CatBoost': CatBoostClassifier(random_state=42)
}

# Collecting results
results = []

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Custom scoring function
scoring = {
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'pr_auc': make_scorer(average_precision_score, needs_proba=True)
}

# Define a scaler
scaler = StandardScaler()

# Iterate over each oversampling technique
for name, oversampler in oversampler_dict.items():
    print(f"Applying oversampling technique: {name}")
    
    # Iterate over each model
    for model_name, model in models_dict.items():
        print(f"Training model: {model_name} with oversampling technique: {name}")
        
        # Set up the pipeline
        if not oversampler:
            pipeline = model
        else:
            pipeline = make_pipeline(
                scaler,
                oversampler,
                model
            )
        
        # Perform cross-validation
        cv_results = cross_validate(
            pipeline, 
            X_train, 
            y_train, 
            scoring=scoring, 
            cv=cv, 
            return_train_score=False
        )
        
        # Print cross-validation results
        print(f"{model_name} results:")
        print(
            'ROC-AUC: {0:.3f} +/- {1:.3f}'.format(
            cv_results['test_roc_auc'].mean(), cv_results['test_roc_auc'].std()
            )
        )
        print(
            'PR-AUC: {0:.3f} +/- {1:.3f}'.format(
            cv_results['test_pr_auc'].mean(), cv_results['test_pr_auc'].std()
            )
        )
        print()  # Add a blank line for better readability
        
        # Append the results
        results.append({
            'Oversampling Technique': name,
            'Model': model_name,
            'ROC-AUC': cv_results['test_roc_auc'].mean(),
            'ROC-AUC-std': cv_results['test_roc_auc'].std(),
            'PR-AUC': cv_results['test_pr_auc'].mean(),
            'PR-AUC-std': cv_results['test_pr_auc'].std()
        })

# Create a DataFrame from the results
df_results_2 = pd.DataFrame(results)

# Print the DataFrame as a table
print(df_results_2)

Applying oversampling technique: random
Training model: CatBoost with oversampling technique: random
Learning rate set to 0.091842
0:	learn: 0.6857900	total: 244ms	remaining: 4m 3s
1:	learn: 0.6789649	total: 361ms	remaining: 3m
2:	learn: 0.6731753	total: 470ms	remaining: 2m 36s
3:	learn: 0.6677213	total: 575ms	remaining: 2m 23s
4:	learn: 0.6630264	total: 688ms	remaining: 2m 16s
5:	learn: 0.6584334	total: 794ms	remaining: 2m 11s
6:	learn: 0.6542768	total: 901ms	remaining: 2m 7s
7:	learn: 0.6502164	total: 1.01s	remaining: 2m 5s
8:	learn: 0.6463251	total: 1.12s	remaining: 2m 3s
9:	learn: 0.6424998	total: 1.22s	remaining: 2m
10:	learn: 0.6389375	total: 1.31s	remaining: 1m 58s
11:	learn: 0.6354886	total: 1.43s	remaining: 1m 57s
12:	learn: 0.6320383	total: 1.57s	remaining: 1m 59s
13:	learn: 0.6287672	total: 1.68s	remaining: 1m 57s
14:	learn: 0.6256386	total: 1.79s	remaining: 1m 57s
15:	learn: 0.6226534	total: 1.89s	remaining: 1m 56s
16:	learn: 0.6196471	total: 1.99s	remaining: 1m 54s
17:	lea

C:\Users\dziya\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:363: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Learning rate set to 0.091842
0:	learn: 0.6857069	total: 113ms	remaining: 1m 52s
1:	learn: 0.6793202	total: 227ms	remaining: 1m 53s
2:	learn: 0.6729264	total: 338ms	remaining: 1m 52s
3:	learn: 0.6670793	total: 464ms	remaining: 1m 55s
4:	learn: 0.6612870	total: 588ms	remaining: 1m 56s
5:	learn: 0.6558789	total: 707ms	remaining: 1m 57s
6:	learn: 0.6507391	total: 821ms	remaining: 1m 56s
7:	learn: 0.6457063	total: 935ms	remaining: 1m 55s
8:	learn: 0.6409149	total: 1.06s	remaining: 1m 56s
9:	learn: 0.6362754	total: 1.19s	remaining: 1m 57s
10:	learn: 0.6317575	total: 1.31s	remaining: 1m 57s
11:	learn: 0.6272999	total: 1.43s	remaining: 1m 57s
12:	learn: 0.6230253	total: 1.55s	remaining: 1m 57s
13:	learn: 0.6187875	total: 1.67s	remaining: 1m 57s
14:	learn: 0.6146227	total: 1.78s	remaining: 1m 56s
15:	learn: 0.6105697	total: 1.9s	remaining: 1m 56s
16:	learn: 0.6066418	total: 2.01s	remaining: 1m 56s
17:	learn: 0.6027241	total: 2.13s	remaining: 1m 55s
18:	learn: 0.5989939	total: 2.23s	remaining: 

C:\Users\dziya\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\base.py:363: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Learning rate set to 0.091842
0:	learn: 0.6857900	total: 128ms	remaining: 2m 7s
1:	learn: 0.6788767	total: 259ms	remaining: 2m 9s
2:	learn: 0.6725362	total: 391ms	remaining: 2m 10s
3:	learn: 0.6667082	total: 537ms	remaining: 2m 13s
4:	learn: 0.6609558	total: 659ms	remaining: 2m 11s
5:	learn: 0.6550923	total: 792ms	remaining: 2m 11s
6:	learn: 0.6500044	total: 921ms	remaining: 2m 10s
7:	learn: 0.6449339	total: 1.04s	remaining: 2m 8s
8:	learn: 0.6400576	total: 1.14s	remaining: 2m 5s
9:	learn: 0.6352496	total: 1.26s	remaining: 2m 4s
10:	learn: 0.6306387	total: 1.37s	remaining: 2m 3s
11:	learn: 0.6259766	total: 1.49s	remaining: 2m 2s
12:	learn: 0.6215984	total: 1.61s	remaining: 2m 2s
13:	learn: 0.6171915	total: 1.72s	remaining: 2m 1s
14:	learn: 0.6129839	total: 1.83s	remaining: 1m 59s
15:	learn: 0.6089033	total: 1.93s	remaining: 1m 58s
16:	learn: 0.6047102	total: 2.04s	remaining: 1m 58s
17:	learn: 0.6008485	total: 2.15s	remaining: 1m 57s
18:	learn: 0.5969111	total: 2.28s	remaining: 1m 57s
1

...

Learning rate set to 0.091842
0:	learn: 0.6866035	total: 137ms	remaining: 2m 16s
1:	learn: 0.6809934	total: 273ms	remaining: 2m 16s
2:	learn: 0.6759784	total: 407ms	remaining: 2m 15s
3:	learn: 0.6715069	total: 528ms	remaining: 2m 11s
4:	learn: 0.6672976	total: 649ms	remaining: 2m 9s
5:	learn: 0.6633503	total: 794ms	remaining: 2m 11s
6:	learn: 0.6593129	total: 896ms	remaining: 2m 7s
7:	learn: 0.6556144	total: 1.02s	remaining: 2m 6s
8:	learn: 0.6519776	total: 1.14s	remaining: 2m 5s
9:	learn: 0.6484661	total: 1.24s	remaining: 2m 3s
10:	learn: 0.6450876	total: 1.36s	remaining: 2m 2s
11:	learn: 0.6419147	total: 1.49s	remaining: 2m 2s
12:	learn: 0.6386499	total: 1.6s	remaining: 2m 1s
13:	learn: 0.6354695	total: 1.72s	remaining: 2m 1s
14:	learn: 0.6325787	total: 1.83s	remaining: 2m
15:	learn: 0.6296340	total: 1.95s	remaining: 1m 59s
16:	learn: 0.6266251	total: 2.07s	remaining: 1m 59s
17:	learn: 0.6239421	total: 2.19s	remaining: 1m 59s
18:	learn: 0.6211600	total: 2.31s	remaining: 1m 59s
19:	le

C:\Users\dziya\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\filter.py:197: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Learning rate set to 0.091842
0:	learn: 0.6869674	total: 120ms	remaining: 2m
1:	learn: 0.6813723	total: 244ms	remaining: 2m 1s
2:	learn: 0.6760958	total: 360ms	remaining: 1m 59s
3:	learn: 0.6715050	total: 486ms	remaining: 2m 1s
4:	learn: 0.6672062	total: 600ms	remaining: 1m 59s
5:	learn: 0.6631978	total: 715ms	remaining: 1m 58s
6:	learn: 0.6593609	total: 834ms	remaining: 1m 58s
7:	learn: 0.6555506	total: 952ms	remaining: 1m 58s
8:	learn: 0.6520144	total: 1.07s	remaining: 1m 58s
9:	learn: 0.6485539	total: 1.19s	remaining: 1m 57s
10:	learn: 0.6452059	total: 1.31s	remaining: 1m 58s
11:	learn: 0.6418908	total: 1.43s	remaining: 1m 57s
12:	learn: 0.6386804	total: 1.54s	remaining: 1m 57s
13:	learn: 0.6353154	total: 1.67s	remaining: 1m 57s
14:	learn: 0.6324347	total: 1.79s	remaining: 1m 57s
15:	learn: 0.6295428	total: 1.9s	remaining: 1m 57s
16:	learn: 0.6263296	total: 2.02s	remaining: 1m 56s
17:	learn: 0.6233534	total: 2.13s	remaining: 1m 56s
18:	learn: 0.6204094	total: 2.23s	remaining: 1m 55s

C:\Users\dziya\anaconda3\Lib\site-packages\imblearn\over_sampling\_smote\filter.py:197: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


Learning rate set to 0.091842
0:	learn: 0.6868254	total: 137ms	remaining: 2m 16s
1:	learn: 0.6811750	total: 265ms	remaining: 2m 12s
2:	learn: 0.6761897	total: 405ms	remaining: 2m 14s
3:	learn: 0.6716328	total: 520ms	remaining: 2m 9s
4:	learn: 0.6672809	total: 664ms	remaining: 2m 12s
5:	learn: 0.6631501	total: 787ms	remaining: 2m 10s
6:	learn: 0.6590951	total: 924ms	remaining: 2m 11s
7:	learn: 0.6554696	total: 1.04s	remaining: 2m 8s
8:	learn: 0.6517538	total: 1.16s	remaining: 2m 7s
9:	learn: 0.6482539	total: 1.27s	remaining: 2m 5s
10:	learn: 0.6448948	total: 1.39s	remaining: 2m 4s
11:	learn: 0.6416694	total: 1.5s	remaining: 2m 3s
12:	learn: 0.6383395	total: 1.63s	remaining: 2m 3s
13:	learn: 0.6349144	total: 1.75s	remaining: 2m 3s
14:	learn: 0.6318935	total: 1.86s	remaining: 2m 2s
15:	learn: 0.6289307	total: 2s	remaining: 2m 2s
16:	learn: 0.6259725	total: 2.11s	remaining: 2m 1s
17:	learn: 0.6229927	total: 2.24s	remaining: 2m 2s
18:	learn: 0.6201529	total: 2.38s	remaining: 2m 3s
19:	learn

### Export classification metrics

In [17]:
# Export results to CSV files
df_results_2.to_csv('7_evaluation_results_oversampling_and_cross_validation.csv', index=False)